<a href="https://colab.research.google.com/github/DenisKai7/GroUp/blob/ml-training/GroUp_stunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dataset dari GITHUB

In [1]:
!wget https://github.com/DenisKai7/GroUp/blob/1632d4fbdc7377e667770bfc4754da79b827dd9d/Stunting_Dataset.csv

--2024-11-06 12:19:48--  https://github.com/DenisKai7/GroUp/blob/1632d4fbdc7377e667770bfc4754da79b827dd9d/Stunting_Dataset.csv
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Stunting_Dataset.csv’

Stunting_Dataset.cs     [ <=>                ] 688.27K  --.-KB/s    in 0.04s   

2024-11-06 12:19:48 (16.4 MB/s) - ‘Stunting_Dataset.csv’ saved [704786]



In [2]:
!wget https://github.com/DenisKai7/GroUp/blob/df39424862ae2fd7a9923085ef97bdf0746cbd26/nutrition.csv

--2024-11-06 12:19:53--  https://github.com/DenisKai7/GroUp/blob/df39424862ae2fd7a9923085ef97bdf0746cbd26/nutrition.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘nutrition.csv’

nutrition.csv           [  <=>               ] 306.82K  1.07MB/s    in 0.3s    

2024-11-06 12:19:54 (1.07 MB/s) - ‘nutrition.csv’ saved [314182]



Import Library yang dibutuhkan

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

untuk menyimpan model

In [5]:
import joblib

untuk menyimpan rekomendasi dalam format JSON

In [6]:
import json

untuk Sistem Rekomendasi

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

untuk pengolahan data waktu

In [8]:
from datetime import datetime, timedelta

untuk tensorflow lite

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

untuk visualisasi

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

untuk evaluasi model

In [11]:
from sklearn.metrics import mean_squared_error

untuk menyimpan model tensorflow

In [12]:
import os